In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
#载入数据
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
#清空默认图，使jupyter可重复调试
tf.reset_default_graph()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
n_input = 28
max_time = 28
#隐层个数
lstm_size = 100
#输出序列长度
n_class = 10
#每批样本数
batch_size = 80
#批次
n_batch_size = mnist.train.num_examples // batch_size

In [3]:
#创建占位符
x = tf.placeholder(tf.float32,[None,28*28])
y = tf.placeholder(tf.float32,[None,10])

In [4]:
#初始化权重
weight = tf.Variable(tf.truncated_normal([lstm_size,n_class],stddev = 0.1))
#初始化偏置
biases = tf.Variable(tf.constant(0.1,shape=[n_class]))

In [5]:
#rnn网络结构
def rnn(X,weights,biases):
    #原始数据的shape为[batch_size,28*28]
    #input = [batch_size, max_time, n_input]
    input_ = tf.reshape(X,[-1,max_time,n_input])
    # 定义LSTM的基本单元
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    #执行多步
    # final_state[0] 是cell state
    # final_state[1] 是hidden stat
    outputs,final_state = tf.nn.dynamic_rnn(lstm_cell,input_,dtype=tf.float32)
    results = tf.nn.softmax(tf.matmul(final_state[1],weights)+biases,1)
    return results

In [6]:

prediction = rnn(x,weight,biases)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))


In [7]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(6):
        for batch in range(n_batch_size):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x: batch_xs,y: batch_ys})
            if(batch%100==0):
                print(str(batch)+"/" + str(n_batch_size))
        acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
        print("Iter" + str(epoch) + " ,Testing Accuracy = " + str(acc))

0/687
100/687
200/687
300/687
400/687
500/687
600/687
Iter0 ,Testing Accuracy = 0.6433
0/687
100/687
200/687
300/687
400/687
500/687
600/687
Iter1 ,Testing Accuracy = 0.7268
0/687
100/687
200/687
300/687
400/687
500/687
600/687
Iter2 ,Testing Accuracy = 0.8154
0/687
100/687
200/687
300/687
400/687
500/687
600/687
Iter3 ,Testing Accuracy = 0.8322
0/687
100/687
200/687
300/687
400/687
500/687
600/687
Iter4 ,Testing Accuracy = 0.9003
0/687
100/687
200/687
300/687
400/687
500/687
600/687
Iter5 ,Testing Accuracy = 0.919
